In [1]:
import pandas as pd
import os

In [2]:
print(os.listdir())

['brain_extraction.ipynb', 'cohort-sublists', 'mri_subjects_updated.txt', 'MRI preprocessing', 'cohort-lists', 'analysis_tsv_data.ipynb', '.DS_Store', 'analysis_s3_derivatives.ipynb', 'targets', 'mri_subjects.txt', 'mri_subjects_exclude.txt', 's3 tsv data', 'phenotypic data', 'subject_data_preprocessed_set.csv', 'subject_data_combined_subset.csv', 'analysis_phenotypic_data.ipynb', 'other', 'post_preprocessing_subject_consolidation.ipynb', 'fs_features', 'excalidraw', 'analysis_subjects_taking_long.ipynb', 's3_subjects.csv', 'ids_classification.txt', 'rclone download', 'contents structure thesis.txt', 'slides', 'preprocessed_subjects.csv', 'paper notes.txt', 'statistical analysis', 'subject_data_regression_subset.csv', 'subject_data_classification_subset.csv', 'freesurfer_stats_retrieval.sh', 'fs_stats', 'neuro venv', 'ids_regression.txt', 'ids_taking_long.txt', 'freesurfer_feature_retrieval.ipynb', 'preprocessing_report.txt', 'ids_preprocessed.txt', 'post_preprocessing_subject_analysis

In [3]:
os.chdir('fs_stats')
#os.listdir()

In [4]:
aseg_dict = {}
global_dict = {}
aparc_dict = {}

for file in os.listdir():
    if file == 'aseg_asegstats_global.csv':
        global_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")
    if (file.endswith('.csv') & file.startswith('aseg')) & (file != 'aseg_asegstats_global.csv'):
        aseg_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")
    if file.endswith('.csv') & (file.startswith('lh') |file.startswith('rh') ) :
        aparc_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")

print(len(aseg_dict), len(global_dict), len(aparc_dict))

7 1 18


In [5]:
# Drop columns in aseg_asegstats_global frame

# Curated
#columns2drop = ['CortexVol', 'CerebralWhiteMatterVol', 'BrainSegVol-to-eTIV', 'MaskVol-to-eTIV', 'SurfaceHoles']
#aseg_dict["aseg_asegstats_global"].drop(columns2drop, axis=1, inplace=True)

In [6]:
# Change column naming pattern for dataframe global
global_label_mapping = {}
global_frame = global_dict['aseg_asegstats_global']

for col in global_frame.columns[1:]:
    global_label_mapping[col] = "global_" + col.lower()

#print(global_label_mapping)

global_frame.rename(global_label_mapping, axis="columns", inplace=True)
global_dict['aseg_asegstats_global'] = global_frame

In [7]:
# Change column naming pattern for all aseg dataframes

for frame in aseg_dict.values():
    label_mapping = {}

    for col in frame.columns[1:]:

        label_mapping[col] = "aseg_" + col.lower() + "_" + frame.columns[0].split(":")[1]
    
    #print(label_mapping)
    frame.rename(label_mapping, axis="columns", inplace=True)

In [8]:
# Change column naming pattern for all aparc dataframes

for frame in aparc_dict.values():
    label_mapping = {}

    for col in frame.columns[1:]:
        label_mapping[col] = "aparc_" + col
    
    #print(label_mapping)
    frame.rename(label_mapping, axis="columns", inplace=True)

In [9]:
# Transform dictionaries into lists
global_list = [global_dict['aseg_asegstats_global']]
aseg_list = list(aseg_dict.values())
aparc_list = list(aparc_dict.values())

In [10]:
## Construct new dataframe with subject-ids only

# Create new dataframe with id column
df_base = pd.DataFrame()
df_base["ID"] = aseg_list[0].iloc[:,0]

# Remove 'sub' substring from id string
def remove_sub(identifier):
    id = identifier.split('-')
    return id[1].strip() 

df_base["ID"] = df_base["ID"].apply(remove_sub)
#df_base

In [11]:
# Delete first column (holding ids) in all dataframes from list

df_list = global_list + aseg_list + aparc_list

for frame in df_list:
    frame.drop(columns=frame.columns[0], axis=1, inplace=True)

In [12]:
# Aggregate frames to single dataframe

joined_list = [df_base] + df_list
df = pd.concat(joined_list, axis=1)

In [13]:
#df

In [14]:
os.chdir('../fs_features/')
df.to_csv('fs_features_total.csv')